![BCC horizontal (1)](https://github.com/damahindra/AIRNOLOGY-23/assets/105963394/90e95d06-e322-4de7-b8aa-a62970f11521)
# BCC Freya

#### Problem Domain
Prediksi curah hujan memiliki relevansi yang sangat penting dalam berbagai aspek kehidupan, termasuk pertanian, mitigasi bencana, pengelolaan sumber daya air, dan perencanaan infrastruktur. Dengan memiliki informasi yang akurat tentang kapan, seberapa banyak, dan di mana hujan akan terjadi, kita dapat mengoptimalkan penggunaan sumber daya pertanian, menghindari kerugian akibat banjir, mengatur pasokan air yang efisien, dan merencanakan pembangunan kota yang tahan cuaca. Dalam konteks pemindahan ibu kota baru Indonesia, prediksi curah hujan yang baik akan membantu dalam perencanaan dan pengaturan jadwal proyek, mengurangi risiko penundaan, dan memastikan kelancaran proyek infrastruktur yang sangat penting tersebut.
#### Diagram Alir Metodologi
![planning AIRNOLOGY drawio (3)](https://github.com/damahindra/AIRNOLOGY-23/assets/105963394/3407cbaf-9e2e-4aae-b66f-301907037e5b)

Proses dimulai dengan pemerolehan data dari Kaggle yang telah disiapkan oleh panitia. Selanjutnya, data tersebut akan mengalami ekstraksi fitur agar nilai-nilainya dapat digunakan. Langkah berikutnya adalah melakukan EDA untuk mendapatkan wawasan dan ide-ide potensial untuk feature engineering. Setelah itu, akan dilakukan tiga skenario feature engineering secara paralel. Setiap skenario akan menghasilkan fitur-fitur yang berbeda untuk meningkatkan ketepatan prediksi. Setelah proses feature engineering selesai, model fitting dan pelatihan akan dilakukan pada masing-masing skenario. Evaluasi model akan dilakukan dengan membagi dataset menjadi data pelatihan dan data validasi menggunakan data split, serta dengan menggunakan cross-validation untuk memastikan keandalan model. Hasil prediksi akan disubmit ke Kaggle untuk pengujian. Score terbaik akan dicatat dan dimasukkan ke dalam suatu knowledge base yang berisi kumpulan ide-ide feature engineering yang telah terbukti efektif dalam menurunkan Root Mean Square Error (RMSE). Proses ini akan berjalan secara berkelanjutan, diulang, dan diperbaiki seiring berjalannya waktu untuk terus meningkatkan kualitas prediksi curah hujan.

# Importing Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Basic Library
import warnings
import os
import time
import numpy as np
np.random.seed(7)
import pandas as pd
import statistics
import math
from timeit import default_timer as timer

# Preprocessing Library
import vtreat
from scipy.stats.mstats import winsorize

# Visualization Library
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import missingno as mno


# Machine Learning Library
from fast_ml.model_development import train_valid_test_split
from catboost import CatBoostRegressor
import pickle

# Maths & Stats
from scipy import stats
from scipy.fft import fft
from scipy.signal import find_peaks
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf

# Model Evaluation Library
from sklearn.metrics import mean_squared_error

# Library setting
sns.set_theme(style="whitegrid")
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Data Extraction

In [ ]:
TRAIN_PATH = "../../datasets/train.csv"
TEST_PATH = "../../datasets/test.csv"
SAMPLE_SUBMISSION_PATH = "../../datasets/sample_submission.csv"

In [ ]:
# Read the CSV file into a DataFrame
train_dat = pd.read_csv(TRAIN_PATH)
test_dat = pd.read_csv(TEST_PATH)

# Preparing features for further analysis and processing

# method for formatting temp, d_point, max_temp, min_temp, and feels
def prep_temp(temp):
    """
    Method untuk membersihkan dan mengesktrak data suhu yang memiliki satuan ukur (contoh : "36.5 Celcius")

    Args:
        temp (str): Data suhu yang memiliki satuan ukur.

    Returns:
        str: Data suhu yang telah dibersihkan satuan ukurnya.

    Examples:
        >>> prep_temp("24.3 C")
        "24.3"
        >>> prep_temp("35 Celcius")
        "35"
    """
    if isinstance(temp, str) :
        temp = temp.replace("Celcius", "")
        temp = temp.replace("C", "")
        temp = temp.replace("°", "")
        temp = temp.strip()
    return temp

# method for formatting prssr 
def prep_prssr(prssr):
    """
    Method untuk membersihkan dan mengesktrak data tekanan yang memiliki satuan ukur (contoh : "36.5 hPa")

    Args:
        prssr (str): Data tekanan yang memiliki satuan ukur.

    Returns:
        str: Data tekanan yang telah dibersihkan satuan ukurnya.

    Examples:
        >>> prep_prssr("45 hPa")
        "45"
        >>> prep_prssr("20 hPa.")
        "20"
    """
    if isinstance(prssr, str):
        prssr = prssr.replace("hPa.", "")
        prssr = prssr.replace("hPa", "")
        prssr = prssr.strip()
    return prssr

# method for formatting hum 
def prep_hum(hum):
    """
    Method untuk membersihkan dan mengesktrak data kelembaban yang memiliki satuan ukur (contoh : "36.5 %")

    Args:
        temp (str): Data kelembaban yang memiliki satuan ukur.

    Returns:
        str: Data kelembaban yang telah dibersihkan satuan ukurnya.

    Examples:
        >>> prep_hum("21%")
        "21"
        >>> prep_hum("30.4 %")
        "30.4"
    """
    if isinstance(hum, str):
        hum = hum.replace("%", "")
        hum = hum.strip()
    return hum

# method for formatting wind_spd and wind_deg
def prep_wind(wind):
    """
    Method untuk membersihkan dan mengesktrak data angin yang memiliki satuan ukur (contoh : "36.5 m/s")

    Args:
        wind (str): Data angin yang memiliki satuan ukur.

    Returns:
        str: Data angin yang telah dibersihkan satuan ukurnya.

    Examples:
        >>> prep_wind("19 m/s")
        "19"
        >>> prep_wind("10.2°")
        "10.2"
    """
    if isinstance(wind, str):
        wind = wind.replace("m/s", "")
        wind = wind.replace("°", "")
        wind = wind.strip()
    return wind

# method for formatting rain_1h
def prep_rain_1h(rain):
    """
    Method untuk membersihkan dan mengesktrak data curah hujan 1 jam lalu yang memiliki satuan ukur (contoh : "36.5 mm")

    Args:
        rain (str): Data curah hujan 1 hari lalu yang memiliki satuan ukur.

    Returns:
        str: Data curah hujan 1 hari lalu yang telah dibersihkan satuan ukurnya.

    Examples:
        >>> prep_rain_1h("19mm")
        "19"
        >>> prep_rain_1h("zero")
        "0"
    """
    if isinstance(rain, str):
        rain = rain.replace("mm", "")
        rain = rain.replace("zero", "0")
        rain = rain.replace(" ", "0")
        rain = rain.strip()
        try:
            float(rain)
            return rain
        except:
            new_rain = 0.0
            return new_rain
    return rain

# method for formatting rain_3h
def prep_rain_3h(rain):
    """
    Method untuk membersihkan dan mengesktrak data curah hujan 3 jam lalu yang memiliki satuan ukur (contoh : "36.5 mm")

    Args:
        rain (str): Data curah hujan 3 hari lalu yang memiliki satuan ukur.

    Returns:
        str: Data curah hujan 3 hari lalu yang telah dibersihkan satuan ukurnya.

    Examples:
        >>> prep_rain_3h("39mm")
        "39"
        >>> prep_rain_3h("no-rain)
        "0"
    """
    if isinstance(rain, str):
        rain = rain.replace("milimeter", "")
        rain = rain.replace("mm", "")
        rain = rain.replace("no-rain", "0")
        rain = rain.replace("volume:0", "0")
        rain = rain.replace("nol", "0")
        rain = rain.replace("no_rain", "0")
        rain = rain.replace("volume:zero", "0")
        rain = rain.replace("zero", "0")
        rain = rain.strip()
    return rain

# method for formatting snow_1h and snow_3h
def prep_snow(snow):
    """
    Method untuk membersihkan dan mengesktrak data curah salju 1 dan 3 jam lalu yang memiliki satuan ukur (contoh : "36.5 mm")

    Args:
        snow (str): Data curah salju 1 hari lalu yang memiliki satuan ukur.

    Returns:
        str: Data curah salju 1 hari lalu yang telah dibersihkan satuan ukurnya.

    Examples:
        >>> prep_snow_1h("19mm")
        "19"
        >>> prep_snow_1h("no-snow")
        "0"
    """
    if isinstance(snow, str):
        snow = snow.replace("milimeter", "")
        snow = snow.replace("mm", "")
        snow = snow.replace("no-snow", "0")
        snow = snow.replace("volume:0", "0")
        snow = snow.replace("nol", "0")
        snow = snow.replace("no_snow", "0")
        snow = snow.replace("volume:zero", "0")
        snow = snow.replace("zero", "0")
        snow = snow.strip()
    return snow

# method for formatting clouds
def prep_clouds(clouds):
    """
    Method untuk membersihkan dan mengesktrak data persentase awan yang memiliki satuan ukur (contoh : "36.5 %")

    Args:
        clouds (str): Datapersentase awan yang memiliki satuan ukur.

    Returns:
        str: Datapersentase awan yang telah dibersihkan satuan ukurnya.

    Examples:
        >>> prep_clouds_3h("39 %")
        "39"
        >>> prep_clouds_3h("75.2%")
        "75.2"
    """
    if isinstance(clouds, str):
        clouds = clouds.replace("%", "")
        clouds = clouds.strip()
    return clouds

# Engineering categorized features

# method for categorizing hour
def categorize_hour(hour) :
    """
    Method untuk mengkategorikan jam menjadi 8 kategori yang berbeda berdasarkan nilai yang diberikan.

    Args:
        hour (str): data jam yang ingin dikategorikan.

    Returns:
        str: Kategori dari data jam.
            - 'dawn', apabila waktu menunjukkan pukul 4 atau pukul 5
            - 'early morning', apabila waktu menunjukkan pukul 6 atau pukul 7
            - 'late morning', apabila waktu menunjukkan pukul 8, pukul 9, atau pukul 10
            - 'noon', apabila waktu menunjukkan pukul 11, pukul 12, atau pukul 13
            - 'afternoon', apabila waktu menunjukkan pukul 14, pukul 15, atau pukul 16
            - 'evening', apabila waktu menunjukkan pukul 17, pukul 18, atau pukul 19
            - 'night', apabila waktu menunjukkan pukul 20, pukul 21, atau pukul 22
            - 'midnight', apabila waktu menunjukkan pukul 23, pukul 0, pukul 1, pukul 2, atau pukul 3

    Examples:
        >>> categorize_hour(4)
        "dawn"
        >>> categorize_hour(17)
        "evening"
    """
    if hour in [4,5]:
        return "dawn"
    elif hour in [6,7]:
        return "early morning"
    elif hour in [8,9,10]:
        return "late morning"
    elif hour in [11,12,13]:
        return "noon"
    elif hour in [14,15,16]:
        return "afternoon"
    elif hour in [17, 18,19]:
        return "evening"
    elif hour in [20, 21, 22]:
        return "night"
    elif hour in [23,0,1,2,3]:
        return "midnight"

# method for categorizing months in 1 year into quarters 
def categorize_quarter(month) :
    """
    Method untuk mengkategorikan bulan menjadi 4 kategori yang berbeda berdasarkan nilai yang diberikan.

    Args:
        month (str): data bulan yang ingin dikategorikan.

    Returns:
        str: Kategori dari data bulan.
            - 'quarter 1', apabila bulan pada hari itu adalah bulan 1 (Januari), 2 (Februari), atau 3 (Maret)
            - 'quarter 2', apabila bulan pada hari itu adalah bulan 4 (April), 5 (Mei), atau 6 (Juni)
            - 'quarter 3', apabila bulan pada hari itu adalah bulan 7 (Juli), 8 (Agustus), atau 9 (September)
            - 'quarter 4', apabila bulan pada hari itu adalah bulan 10 (Oktober), 11 (November), atau 12 (Desember)

    Examples:
        >>> categorize_quarter(4)
        "quarter 2"
        >>> categorize_quarter(10)
        "quarter 4"
    """
    if month in [1,2,3]:
        return "quarter 1"
    elif month in [4,5,6]:
        return "quarter 2"
    elif month in [7,8,9]:
        return "quarter 3"
    elif month in [10,11,12]:
        return "quarter 4"

# Date and time extraction

# datetime -> dihapus karena redundant
train_dat.drop(columns=["datetime"], inplace=True)
test_dat.drop(columns=["datetime"], inplace=True)

# convert the 'timestamp' column to a datetime object
train_dat['datetime_iso'] = pd.to_datetime(train_dat['datetime_iso'])
test_dat['datetime_iso'] = pd.to_datetime(test_dat['datetime_iso'])

# extract year, month, day, and hour
# train data
train_dat['month'] = train_dat['datetime_iso'].dt.month.astype(float) # extracting month
train_dat['day'] = train_dat['datetime_iso'].dt.day # extracting day
train_dat['hour'] = train_dat['datetime_iso'].dt.hour # extracting hour
train_dat['year'] = train_dat['datetime_iso'].dt.year # extracting year
train_dat['hour_cat'] = train_dat['hour'].apply(categorize_hour) # categorize hour and engineering hour_cat feature
train_dat['quart_cat'] = train_dat['month'].apply(categorize_quarter) # categorize quarters and engineering quart_cat feature

train_dat['day_name'] = train_dat['datetime_iso'].dt.day_name() # extracting day name

train_dat['weekday'] = train_dat['day_name'].apply(lambda x: 0 if x in ['Saturday', 'Sunday'] else 1) # engineering weekday feature
train_dat['weekend'] = train_dat['day_name'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0) # engineering weekend feature
train_dat['dayofweek'] = train_dat['datetime_iso'].dt.dayofweek # extracting day of week
train_dat.drop(columns=["day_name"], inplace=True) # drop day_name feature

# test data
test_dat['month'] = test_dat['datetime_iso'].dt.month.astype(float) # extracting month
test_dat['day'] = test_dat['datetime_iso'].dt.day # extracting day
test_dat['hour'] = test_dat['datetime_iso'].dt.hour # extracting hour
test_dat['year'] = test_dat['datetime_iso'].dt.year # extracting year
test_dat['hour_cat'] = test_dat['hour'].apply(categorize_hour) # categorize hour and engineering hour_cat feature
test_dat['quart_cat'] = test_dat['month'].apply(categorize_quarter) # categorize quarters and engineering quart_cat feature

test_dat['day_name'] = test_dat['datetime_iso'].dt.day_name() # extracting day name

test_dat['weekday'] = test_dat['day_name'].apply(lambda x: 0 if x in ['Saturday', 'Sunday'] else 1) # engineering weekday feature
test_dat['weekend'] = test_dat['day_name'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0) # engineering weekend feature
test_dat['dayofweek'] = test_dat['datetime_iso'].dt.dayofweek # extracting day of week
test_dat.drop(columns=["day_name"], inplace=True) # drop day_name feature

# dropping datetime_iso feature
train_dat.drop(columns=["datetime_iso"], inplace=True) # dropping datetime_iso on train
test_dat.drop(columns=["datetime_iso"], inplace=True) # dropping datetime_iso on test

# time-zone -> dihapus karena data bukan unique
train_dat.drop(columns=["time-zone"], inplace=True) # dropping time-zone on train
test_dat.drop(columns=["time-zone"], inplace=True) # dropping time-zone on test

preped_train = train_dat.copy() # making new copy of extracted train data called preped_train
preped_test = test_dat.copy() # making new copy of extracted test data called preped_test

# Prepping temp column --> including temp, d_point, feels, min_temp, max_temp
for column in ['temp','d_point','feels','min_temp','max_temp']:
    preped_train[column] = preped_train[column].apply(lambda x: prep_temp(x))
    preped_test[column] = preped_test[column].apply(lambda x: prep_temp(x))

# Prepping prssr column
for column in ['prssr']:
    preped_train[column] = preped_train[column].apply(lambda x:prep_prssr(x))
    preped_test[column] = preped_test[column].apply(lambda x:prep_prssr(x))

# Prepping hum column
for column in ['hum']:
    preped_train[column] = preped_train[column].apply(lambda x:prep_hum(x))
    preped_test[column] = preped_test[column].apply(lambda x:prep_hum(x))

# Prepping wind column --> including wind_spd, wind_deg
for column in ['wind_spd', 'wind_deg']:
    preped_train[column] = preped_train[column].apply(lambda x:prep_wind(x))
    preped_test[column] = preped_test[column].apply(lambda x:prep_wind(x))

# Prepping rain_1h
for column in ['rain_1h']:
    preped_train[column] = preped_train[column].apply(lambda x:prep_rain_1h(x))

# Prepping rain_3h column
for column in ['rain_3h']:
    preped_train[column] = preped_train[column].apply(lambda x:prep_rain_3h(x))
    preped_test[column] = preped_test[column].apply(lambda x:prep_rain_3h(x))

# Prepping snow column --> including snow_1h, snow_3h
for column in ['snow_1h', 'snow_3h']:
    preped_train[column] = preped_train[column].apply(lambda x:prep_snow(x))
    preped_test[column] = preped_test[column].apply(lambda x:prep_snow(x))

# Prepping clunds column
for column in ['clouds']:
    preped_train[column] = preped_train[column].apply(lambda x:prep_clouds(x))
    preped_test[column] = preped_test[column].apply(lambda x:prep_clouds(x))

# engineering month_seasonal feature adjusted from the features stationarity
preped_train['month_seasonal']= np.sin(0.6 * (preped_train['month'] - 1)) + 1
preped_test['month_seasonal']= np.sin(0.6 * (preped_test['month'] - 1)) + 1

preped_train['rain_1h'] = preped_train['rain_1h'].astype(float) # converting rain_1h data type on train data to float

preped_train['wind_deg'] = preped_train['wind_deg'].astype(float) % 360 # adjusted wind_deg data so all rows will be < 360 on train data
preped_test['wind_deg'] = preped_test['wind_deg'].astype(float) % 360 # adjusted wind_deg data so all rows will be < 360on test data

preped_train.drop(['day'], axis=1, inplace=True) # drop day feature on train data
preped_test.drop(['day'], axis=1, inplace=True) # drop day feature on train data

# Changing to `category` data type for normal categorical data
cat_cols = ['visibility', 'sea_level', 'grnd_level', 'hour', 'hour_cat']

preped_train[cat_cols] = preped_train[cat_cols].astype('category')
preped_test[cat_cols] = preped_train[cat_cols].astype('category')

# Changing to `float` data type for numerical data
num_cols_train = ['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h', 'clouds', 'month']
num_cols_test = ['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'wind_deg', 'rain_3h', 'snow_1h', 'snow_3h', 'clouds', 'month']

preped_train[num_cols_train] = preped_train[num_cols_train].astype('float')
preped_test[num_cols_test] = preped_test[num_cols_test].astype('float')

eda_train = preped_train.copy() # making new copy of preped_train called eda_train

# Preprocessing

##### Feature Engineering

In [ ]:
# categorize d_point 
def categorize_d_point(d_point) :
    """
    Kategorisasi d_point berdasarkan nilai yang diberikan.

    Args:
        d_point (float): Nilai titik embun yang akan dikategorikan.

    Returns:
        str: Kategori titik embun yang mewakili tingkatnya.
            - 'low' untuk nilai <= 19.
            - 'moderate' untuk nilai > 19 dan <= 25.
            - 'high' untuk nilai > 25.

    Examples:
        >>> categorize_d_point(15)
        'low'
        >>> categorize_d_point(23)
        'moderate'
        >>> categorize_d_point(30)
        'high'
    """
    if d_point <= 19 :
        return 'low'
    elif d_point > 19 and d_point <= 25 :
        return 'moderate'
    elif d_point > 25 :
        return 'high'

# categorize feels
def categorize_feels(feels) :
    """
    Kategorisasi nilai perasaan (feels) berdasarkan rentang tertentu.

    Args:
        feels (float): Nilai perasaan yang akan dikategorikan.

    Returns:
        str: Kategori perasaan yang mewakili tingkat keamanannya.
            - 'safe' untuk nilai <= 27.
            - 'caution' untuk nilai > 27 dan <= 32.
            - 'danger' untuk nilai > 32 dan <= 41.
            - 'extreme danger' untuk nilai > 41.

    Examples:
        >>> categorize_feels(23)
        'safe'
        >>> categorize_feels(30)
        'caution'
        >>> categorize_feels(35)
        'danger'
        >>> categorize_feels(45)
        'extreme danger'
    """
    if feels <= 27 :
        return 'safe'
    elif feels > 27 and feels <= 32 :
        return 'caution'
    elif feels > 32 and feels <= 41  :
        return 'danger'
    elif feels > 41  :
        return 'extreme danger'

# categorize wind_deg into 8 regions
def categorize_wind_deg(wind_deg) :
    """
    Kategorisasi arah angin berdasarkan nilai derajat.

    Args:
        wind_deg (float): Nilai derajat arah angin (0-360).

    Returns:
        str: Kategori region berdasarkan derajat arah angin.
            - 'region 1' untuk nilai > 0 dan <= 45.
            - 'region 2' untuk nilai > 45 dan <= 90.
            - 'region 3' untuk nilai > 90 dan <= 135.
            - 'region 4' untuk nilai > 135 dan <= 180.
            - 'region 5' untuk nilai > 180 dan <= 225.
            - 'region 6' untuk nilai > 225 dan <= 270.
            - 'region 7' untuk nilai > 270 dan <= 315.
            - 'region 8' untuk nilai > 315 dan <= 360.

    Examples:
        >>> categorize_wind_deg(30)
        'region 1'
        >>> categorize_wind_deg(100)
        'region 3'
        >>> categorize_wind_deg(270)
        'region 6'
        >>> categorize_wind_deg(350)
        'region 8'
    """
    if wind_deg > 0 and wind_deg <= 45 :
        return 'region 1'
    elif wind_deg > 45 and wind_deg <= 90 :
        return 'region 2'
    elif wind_deg > 90 and wind_deg <= 135 :
        return 'region 3'
    elif wind_deg > 135 and wind_deg <= 180 :
        return 'region 4'
    elif wind_deg > 180 and wind_deg <= 225 :
        return 'region 5'
    elif wind_deg > 225 and wind_deg <= 270 :
        return 'region 6'
    elif wind_deg > 270 and wind_deg <= 315 :
        return 'region 7'
    elif wind_deg > 315 and wind_deg <= 360 :
        return 'region 8'
    
# categorize hum
def categorize_hum(hum) :
    """
    Kategorisasi kelembapan udara berdasarkan nilai yang diberikan.

    Args:
        hum (float): Nilai kelembapan udara (0-150).

    Returns:
        str: Kategori kelembapan udara berdasarkan nilai.
            - 'dry' untuk kelembapan kurang dari 30% dari nilai maksimum.
            - 'normal' untuk kelembapan antara 30% hingga kurang dari 60% dari nilai maksimum.
            - 'humid' untuk kelembapan antara 60% hingga kurang dari 80% dari nilai maksimum.
            - 'very humid' untuk kelembapan 80% atau lebih dari nilai maksimum.

    Examples:
        >>> categorize_hum(15)
        'dry'
        >>> categorize_hum(45)
        'normal'
        >>> categorize_hum(90)
        'humid'
        >>> categorize_hum(120)
        'very humid'
    """
    max_hum = 150
    if hum < (0.3*max_hum) :
        return 'dry'
    elif hum >= (0.3*max_hum) and hum < (0.6*max_hum) :
        return 'normal'
    elif hum >= (0.6*max_hum) and hum < (0.8*max_hum) :
        return 'humid'
    elif hum >= (0.8*max_hum) :
        return 'very humid'
    
# categorize wind_spd
def categorize_wind_spd(wind) :
    """
    Kategorisasi kecepatan angin berdasarkan nilai yang diberikan.

    Args:
        wind (float): Nilai kecepatan angin dalam satuan yang sesuai.

    Returns:
        str: Kategori kecepatan angin berdasarkan nilai.
            - 'light air' untuk kecepatan angin kurang dari 1.6.
            - 'light breeze' untuk kecepatan angin antara 1.6 hingga kurang dari 3.4.
            - 'moderate breeze' untuk kecepatan angin antara 3.4 hingga kurang dari 5.1.
            - 'fresh breeze' untuk kecepatan angin antara 5.1 hingga kurang dari 7.5.
            - 'strong breeze' untuk kecepatan angin antara 7.5 hingga kurang dari 10.1.
            - 'near gale' untuk kecepatan angin antara 10.1 hingga kurang dari 13.3.
            - 'gale' untuk kecepatan angin antara 13.3 hingga kurang dari 15.6.
            - 'severe gale' untuk kecepatan angin antara 15.6 hingga kurang dari 18.6.
            - 'storm' untuk kecepatan angin antara 18.6 hingga kurang dari 21.8.
            - 'severe storm' untuk kecepatan angin 21.8 atau lebih.

    Examples:
        >>> categorize_wind_spd(1.0)
        'light air'
        >>> categorize_wind_spd(4.0)
        'moderate breeze'
        >>> categorize_wind_spd(20.0)
        'severe storm'
    """
    if wind < 1.6 :
        return 'light air'
    elif wind >= 1.6 and wind < 3.4 :
        return 'light breeze'
    elif wind >= 3.4 and wind < 5.1 :
        return 'moderate breeze'
    elif wind >= 5.1 and wind < 7.5 :
        return 'fresh breeze'
    elif wind >=7.5 and wind < 10.1 :
        return 'strong breeze'
    elif wind >= 10.1 and wind < 13.3 :
        return 'near gale'
    elif wind >= 13.3 and wind < 15.6 :
        return 'gale'
    elif wind >= 15.6 and wind < 18.6 :
        return 'severe gale'
    elif wind >= 18.6 and wind < 21.8 :
        return 'storm'
    elif wind >= 21.8 :
        return 'severe storm'
    
# feature interactions
def hum_temp(hum, temp):
    """
    Menghitung rasio kelembapan udara terhadap suhu.

    Args:
        hum (int or float): Nilai kelembapan udara dalam persen.
        temp (int or float): Nilai suhu dalam satuan yang sesuai.

    Returns:
        float or None: Rasio kelembapan udara terhadap suhu jika kedua argumen valid,
                       atau None jika salah satu argumen bukan tipe int atau float.
    """
    if isinstance(hum, (int, float)) and isinstance(temp, (int, float)):
        new_feature = hum / temp
    else:
        new_feature = None  # Provide a default value when hum is not a string
    return new_feature


def hum_wind(hum, wind_spd):
    """
    Menghitung rasio kelembapan udara terhadap kecepatan angin.

    Args:
        hum (int or float): Nilai kelembapan udara dalam persen.
        wind_spd (int or float): Nilai kecepatan angin dalam satuan yang sesuai. Harus bukan nol.

    Returns:
        float or None: Rasio kelembapan udara terhadap kecepatan angin jika kedua argumen valid dan wind_spd bukan nol,
                       atau None jika salah satu argumen bukan tipe int atau float, atau wind_spd adalah nol.
    """
    if isinstance(hum, (int, float)) and isinstance(wind_spd, (int, float)) and wind_spd != 0:
        new_feature = hum / wind_spd
    else:
        new_feature = None  # Provide a default value when hum is not a string
    return new_feature

def temp_wind(temp, wind_spd):
    """
    Menghitung rasio suhu terhadap kecepatan angin.

    Args:
        temp (int or float): Nilai suhu dalam satuan yang sesuai.
        wind_spd (int or float): Nilai kecepatan angin dalam satuan yang sesuai. Harus bukan nol.

    Returns:
        float or None: Rasio suhu terhadap kecepatan angin jika kedua argumen valid dan wind_spd bukan nol,
                       atau None jika salah satu argumen bukan tipe int atau float, atau wind_spd adalah nol.
    """
    if isinstance(temp, (int, float)) and isinstance(wind_spd, (int, float)) and wind_spd != 0:
        new_feature = temp / wind_spd
    else:
        new_feature = None  # Provide a default value when hum is not a string
    return new_feature

def interaction_feature(df, col1, col2):
    """
    Menambahkan fitur interaksi antara dua kolom dalam DataFrame.

    Args:
        df (DataFrame): DataFrame yang akan dimodifikasi.
        col1 (str): Nama kolom pertama.
        col2 (str): Nama kolom kedua.

    Returns:
        DataFrame: DataFrame yang telah dimodifikasi dengan penambahan fitur interaksi.
    """
    df[col1 + '_*_' + col2] = df[col1].astype(float) * df[col2].astype(float)
    return df

def feature_nh_ago(df, col, n):
    """
    Menambahkan fitur yang merepresentasikan nilai kolom pada n jam yang lalu dalam DataFrame.

    Args:
        df (DataFrame): DataFrame yang akan dimodifikasi.
        col (str): Nama kolom yang akan digunakan sebagai dasar fitur.
        n (int): Jumlah jam yang akan dihitung ke belakang.

    Returns:
        DataFrame: DataFrame yang telah dimodifikasi dengan penambahan fitur yang merepresentasikan nilai kolom n jam yang lalu.
    """
    df[f'{col}_{n}h_ago'] = df[col].shift(n)
    df[f'{col}_{n}h_ago'] = df[f'{col}_{n}h_ago'].fillna(df[f'{col}_{n}h_ago'].mean())
    return df

def percent_change_with_lag(df, col, n):
    """
    Menghitung persentase perubahan kolom terhadap nilai kolom n jam yang lalu dalam DataFrame.

    Args:
        df (DataFrame): DataFrame yang akan dimodifikasi.
        col (str): Nama kolom yang akan digunakan sebagai dasar perhitungan.
        n (int): Jumlah jam yang akan dihitung ke belakang.

    Returns:
        DataFrame: DataFrame yang telah dimodifikasi dengan penambahan fitur yang menggambarkan persentase perubahan kolom terhadap nilai kolom n jam yang lalu.
    """
    df[f'{col}_percent_change_with_lag_{n}'] = df[col].div(df[f'{col}_{n}h_ago'])
    return df

def moving_average(data, window_size):
    """
    Menghitung rata-rata bergerak dari data menggunakan jendela geser.

    Args:
        data (list or pandas.Series): Data yang akan digunakan untuk menghitung rata-rata bergerak.
        window_size (int): Ukuran jendela (jumlah data) untuk perhitungan rata-rata.

    Returns:
        list: Daftar yang berisi rata-rata bergerak dari data.
    """
    windows = data.rolling(window_size)
    moving_averages = windows.mean()
    moving_averages_list = moving_averages.tolist()
    final_list = moving_averages_list[window_size - 1:]

    # Define the number of zeros to add at the end
    num_zeros_to_add = window_size-1
    # Add the specified number of zero values at the end of the final_list
    final_list.extend([0] * num_zeros_to_add)
    
    return final_list

def moving_var(data, window_size):
    """
    Menghitung variansi bergerak dari data menggunakan jendela geser.

    Args:
        data (list or pandas.Series): Data yang akan digunakan untuk menghitung variansi bergerak.
        window_size (int): Ukuran jendela (jumlah data) untuk perhitungan variansi.

    Returns:
        list: Daftar yang berisi variansi bergerak dari data.
    """
    windows = data.rolling(window_size)
    moving_averages = windows.std()
    moving_averages_list = moving_averages.tolist()
    final_list = moving_averages_list[window_size - 1:]

    # Define the number of zeros to add at the end
    num_zeros_to_add = window_size-1

    # Add the specified number of zero values at the end of the final_list
    final_list.extend([0] * num_zeros_to_add)
    
    return final_list

def cum_mean(df, col) :
    """
    Menghitung rata-rata kumulatif dari kolom dalam DataFrame.

    Args:
        df (DataFrame): DataFrame yang akan dimodifikasi.
        col (str): Nama kolom yang akan digunakan untuk menghitung rata-rata kumulatif.

    Returns:
        DataFrame: DataFrame yang telah dimodifikasi dengan penambahan kolom rata-rata kumulatif.
    """
    df[f'{col}_cumulative_mean'] = df[col].expanding().mean()
    return df

def EMAs(df, col, degree):
    """
    Menghitung Exponential Moving Average (EMA) dari kolom dalam DataFrame.

    Args:
        df (DataFrame): DataFrame yang akan dimodifikasi.
        col (str): Nama kolom yang akan digunakan untuk menghitung EMA.
        degree (int): Derajat EMA yang akan dihitung.

    Returns:
        DataFrame: DataFrame yang telah dimodifikasi dengan penambahan kolom EMA.

    """
    df[col + '_EMA_' + str(degree)] = df[col].ewm(span=degree, adjust=False).mean()
    return df

def rate_of_change(df, col) :
    """
    Menghitung tingkat perubahan (rate of change) dari kolom dalam DataFrame.

    Args:
        df (DataFrame): DataFrame yang akan dimodifikasi.
        col (str): Nama kolom yang akan digunakan untuk menghitung tingkat perubahan.

    Returns:
        DataFrame: DataFrame yang telah dimodifikasi dengan penambahan kolom tingkat perubahan.

    """
    df[f'{col}_roc'] = df[col].diff()
    df[f'{col}_roc'] = df[f'{col}_roc'].fillna(df[f'{col}_roc'].mean())
    return df
    
def accelerate(df, col) :
    """
    Menghitung percepatan (acceleration) dari kolom dalam DataFrame.

    Args:
        df (DataFrame): DataFrame yang akan dimodifikasi.
        col (str): Nama kolom yang akan digunakan untuk menghitung percepatan.

    Returns:
        DataFrame: DataFrame yang telah dimodifikasi dengan penambahan kolom percepatan.

    """
    df[f'{col}_acc'] = df[col].diff().diff()
    df[f'{col}_acc'] = df[f'{col}_acc'].fillna(df[f'{col}_acc'].mean())
    return df

class OccuranceTransformer():
    def last_occurance(self, df, col, mode="train"):
        """
        Menambahkan fitur yang merepresentasikan waktu terakhir munculnya suatu nilai dalam kolom.

        Args:
            df (DataFrame): DataFrame yang akan dimodifikasi.
            col (str): Nama kolom yang akan digunakan untuk menghitung waktu terakhir munculnya nilai.
            mode (str, optional): Mode penggunaan, "train" atau "test". Jika "train", maka nilai unik dalam kolom akan disimpan.

        Returns:
            DataFrame: DataFrame yang telah dimodifikasi dengan penambahan fitur waktu terakhir munculnya nilai dalam kolom.

        """
        if mode == "train":
            self.unique_val = df[col].value_counts().index
        print(f"col: {col} | mode: {mode}")
        for val in self.unique_val:
            print(f"val: {val}")
            last_not_humid_index = -1
            last_occurrences = []
            
            for index, row in df.iterrows():
                if row[col] == val:
                    last_not_humid_index = index
                last_occurrences.append(index - last_not_humid_index)
            df[f'{col}_last_{val}'] = last_occurrences

            pass
        
        return df

# manual feature engineering #1 : cloudy => 1 if yes (>70), 0 if no (<=70) 
preped_train['cloudy'] = preped_train['clouds'].apply(lambda x: 1 if x > 70 else 0)
preped_test['cloudy'] = preped_test['clouds'].apply(lambda x: 1 if x > 70 else 0)

# manual feature engineering #2 : temp_range => max_temp - min_temp
preped_train['temp_range'] = preped_train['max_temp'] - preped_train['min_temp']
preped_test['temp_range'] = preped_test['max_temp'] - preped_test['min_temp']

# execute categorization
preped_train['wind_spd_cat'] = preped_train['wind_spd'].apply(lambda x : categorize_wind_spd(x))
preped_test['wind_spd_cat'] = preped_test['wind_spd'].apply(lambda x : categorize_wind_spd(x))

preped_train['hum_cat'] = preped_train['hum'].apply(lambda x: categorize_hum(x))
preped_test['hum_cat'] = preped_test['hum'].apply(lambda x: categorize_hum(x))

preped_train['wind_deg_cat'] = preped_train['wind_deg'].apply(lambda x: categorize_wind_deg(x))
preped_test['wind_deg_cat'] = preped_test['wind_deg'].apply(lambda x: categorize_wind_deg(x))

preped_train['feels_cat'] = preped_train['feels'].apply(lambda x: categorize_feels(x))
preped_test['feels_cat'] = preped_test['feels'].apply(lambda x: categorize_feels(x))

preped_train['d_point_cat'] = preped_train['d_point'].apply(lambda x: categorize_d_point(x))
preped_test['d_point_cat'] = preped_test['d_point'].apply(lambda x: categorize_d_point(x))

# execute interaction by division (hum with temp, hum with wind_spd, and temp with wind_spd)
preped_train['hum_temp'] = preped_train.apply(
    lambda row: hum_temp(row['hum'], row['temp']), axis=1)
preped_test['hum_temp'] = preped_test.apply(
    lambda row: hum_temp(row['hum'], row['temp']), axis=1)

preped_train['hum_wind'] = preped_train.apply(
    lambda row: hum_wind(row['hum'], row['wind_spd']), axis=1)
preped_test['hum_wind'] = preped_test.apply(
    lambda row: hum_wind(row['hum'], row['wind_spd']), axis=1)

preped_train['temp_wind'] = preped_train.apply(
    lambda row: temp_wind(row['temp'], row['wind_spd']), axis=1)
preped_test['temp_wind'] = preped_test.apply(
    lambda row: temp_wind(row['temp'], row['wind_spd']), axis=1)

# execute interaction method (by multiplication)

# hum, temp, wind_spd, and wind_deg multiplication
preped_train = interaction_feature(preped_train, 'hum', 'temp')
preped_test = interaction_feature(preped_test, 'hum', 'temp')

preped_train = interaction_feature(preped_train, 'hum', 'wind_spd')
preped_test = interaction_feature(preped_test, 'hum', 'wind_spd')

preped_train = interaction_feature(preped_train, 'temp', 'wind_spd')
preped_test = interaction_feature(preped_test, 'temp', 'wind_spd')

preped_train = interaction_feature(preped_train, 'wind_deg', 'wind_spd')
preped_test = interaction_feature(preped_test, 'wind_deg', 'wind_spd')

# multiplication for prssr and clouds feature
for column in ['hum', 'temp', 'wind_spd', 'wind_deg'] :
    preped_train = interaction_feature(preped_train, 'prssr', column)
    preped_test = interaction_feature(preped_test, 'prssr', column)

    preped_train = interaction_feature(preped_train, 'clouds', column)
    preped_test = interaction_feature(preped_test, 'clouds', column)

# dropping visibility, sea_level, grnd_level, and year feature in both train and test data.
preped_train.drop(['visibility', 'sea_level', 'grnd_level',
                  'year'], axis=1, inplace=True)
preped_test.drop(['visibility', 'sea_level', 'grnd_level',
                 'year'], axis=1, inplace=True)

# engineering lag feature in both train and test data.
numeric_col = ['hum', 'temp', 'clouds', 'wind_spd', 'wind_deg', 'prssr', 'd_point', 'feels']
days = [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 21, 22]

for c in numeric_col:
    for d in days :
        # train
        preped_train = feature_nh_ago(preped_train, c, d)
        # test
        preped_test = feature_nh_ago(preped_test, c, d)

for c in numeric_col:
    for d in days :
        # train
        preped_train = percent_change_with_lag(preped_train, c, d)
        # test
        preped_test = percent_change_with_lag(preped_test, c, d)

# engineering moving average, moving variance, EMA (Exponential Moving Average), rate of change, acceleration, cummulative mean, and last occurance feature in both train and test data.
numeric_col = ['hum', 'temp', 'clouds', 'wind_spd', 'wind_deg', 'prssr', 'd_point', 'feels', 'temp_wind', 'hum_wind', 'hum_temp', 'hum_*_temp',
 'hum_*_wind_spd', 'temp_*_wind_spd', 'wind_deg_*_wind_spd','prssr_*_hum','clouds_*_hum','prssr_*_temp','clouds_*_temp','prssr_*_wind_spd','clouds_*_wind_spd','prssr_*_wind_deg','clouds_*_wind_deg']
window_sizes = [2, 3, 4, 5, 6, 7, 14, 21, 28, 30]

# moving average
for c in numeric_col:
    for w in window_sizes:
        preped_train[f'{c}mov_avg{w}'] = moving_average(preped_train[c], w)
        preped_test[f'{c}mov_avg{w}'] = moving_average(preped_test[c], w)

# moving variance
for c in numeric_col:
    for w in window_sizes:
        preped_train[f'{c}mov_var{w}'] = moving_var(preped_train[c], w)
        preped_test[f'{c}mov_var{w}'] = moving_var(preped_test[c], w)

# EMA (Exponential Moving Average)
for c in numeric_col:
    for w in window_sizes:
        preped_train = EMAs(preped_train, c, w)
        preped_test = EMAs(preped_test, c, w)

# rate of change
for c in numeric_col:
    preped_train = rate_of_change(preped_train, c)
    preped_test = rate_of_change(preped_test, c)

# acceleration
for c in numeric_col:
    preped_train = accelerate(preped_train, c)
    preped_test = accelerate(preped_test, c)

# cummulative mean
for c in numeric_col:
    preped_train = cum_mean(preped_train, c)
    preped_test = cum_mean(preped_test, c)

# last occurance
hour_cat = OccuranceTransformer()
preped_train = hour_cat.last_occurance(preped_train, "hour_cat")
preped_test = hour_cat.last_occurance(preped_test, "hour_cat", mode="test")

col: hour_cat | mode: train
val: midnight
val: afternoon
val: evening
val: late morning
val: night
val: noon
val: dawn
val: early morning
col: hour_cat | mode: test
val: midnight
val: afternoon
val: evening
val: late morning
val: night
val: noon
val: dawn
val: early morning


#### Data Conditioning

In [ ]:
# Data conditioning menggunakan vtreat
transform = vtreat.NumericOutcomeTreatment(
    outcome_name="rain_1h",
    params=vtreat.vtreat_parameters({
        "missingness_imputation": np.mean, # impute nan dengan mean
        "filter_to_recommended": False
    })
)

transformed_train = transform.fit_transform(preped_train)
transformed_test = transform.transform(preped_test)

MAX_TEMP = 50 # threshold suhu, karena suhu yang terlalu tinggi tidak masuk akal
MAX_HUM = 150 # threshold humidity, karena humidity > 100% memasuki titik embun
MAX_MAX_TEMP = 50 # sama dengan suhu
MAX_FEELS = 40 # threshold feels, feels terlalu tinggi tidak masuk akal
MAX_PRESSURE = 1200 # threshold pressure, pressure terlalu tinggi tidak ditemui di data test
MAX_MIN_TENP = 60 # threshold min temp
MAX_D_POINT = 30 # threshold dew point

print(preped_train.shape)
transformed_train = transformed_train[transformed_train["max_temp"] < MAX_MAX_TEMP]
transformed_train = transformed_train[transformed_train["hum"] < MAX_HUM]
transformed_train = transformed_train[transformed_train["feels"] < MAX_FEELS]
transformed_train = transformed_train[transformed_train["prssr"] < MAX_PRESSURE]
transformed_train = transformed_train[transformed_train["min_temp"] < MAX_MIN_TENP]
transformed_train = transformed_train[transformed_train["temp"] < MAX_TEMP]
transformed_train = transformed_train[transformed_train["d_point"] < MAX_D_POINT]
print(transformed_train.shape)

# menghilangkan data yang rain_1h < 0. Karena curah hujan selalu >= 0
transformed_train = transformed_train[transformed_train["rain_1h"] >= 0]

(341880, 1019)
(307340, 1145)


# EDA (Exploratory Data Analysis)

In [ ]:
# Menyalin data `preped_train` untuk proses EDA
data_viz = eda_train.copy()

# Menghilangkan nilai outlier agar data lebih mudah di analisa
MAX_TEMP = 50
MAX_D_POINT = 30
MAX_FEELS = 40
MAX_HUM = 100
MAX_RAIN_1H = 19
MAX_MIN_TENP = 60
MAX_MAX_TEMP = 50
MAX_PRESSURE = 1200

data_viz = data_viz[data_viz["temp"] < MAX_TEMP]
data_viz = data_viz[data_viz["d_point"] < MAX_D_POINT]
data_viz = data_viz[data_viz["min_temp"] < MAX_MIN_TENP]
data_viz = data_viz[data_viz["max_temp"] < MAX_MAX_TEMP]
data_viz = data_viz[data_viz["hum"] < MAX_HUM]
data_viz = data_viz[data_viz["rain_1h"] < MAX_RAIN_1H]
data_viz = data_viz[data_viz["feels"] < MAX_FEELS]
data_viz = data_viz[data_viz["prssr"] < MAX_PRESSURE]

# Statistik dari kolom numerik
data_viz.select_dtypes(exclude='object').describe().T

In [ ]:
# Fungsi ini digunakan untuk melakukan analisis univariat pada kolom-kolom numerik dalam DataFrame
def univariate_analysis(preped_train, height=10):
    """
    Melakukan analisis univariat pada kolom-kolom numerik dalam DataFrame.

    Parameters:
        data (DataFrame): DataFrame yang akan dianalisis.
        tinggi (int, optional): Tinggi gambar subplot dalam inci (default: 10).

    Returns:
        None

    Contoh Penggunaan:
        analisis_univariat(data_viz, tinggi=10)
    """
    # Memfilter DataFrame untuk hanya mengandung kolom-kolom numerik
    numerical_columns = preped_train.select_dtypes(include=['number'])
    
    # Menentukan jumlah baris dan kolom untuk subplot secara dinamis
    num_numerical_cols = len(numerical_columns.columns)
    num_subplot_cols = 3  # Number of columns for subplots
    
    num_subplot_rows = (num_numerical_cols + num_subplot_cols - 1) // num_subplot_cols 
    
    # Membuat subplot
    plt.figure(figsize=(16, height))
    plt.suptitle('Analisis Univariat Kolom-Kolom Numerik', fontsize=16, fontweight='bold', alpha=0.8, y=1.)
    for i, column in enumerate(numerical_columns.columns):
        plt.subplot(num_subplot_rows, num_subplot_cols, i + 1)
        sns.kdeplot(data=numerical_columns[column])
        plt.title(f'Distribusi {column}')
    
    plt.tight_layout()
    plt.show()

univariate_analysis(data_viz, height=10)

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/analisis_numerik.png?raw=true)

In [ ]:
# Menampilkan analisis univariate untuk kolom kategorikal
plt.figure(figsize=(16, 8))
plt.suptitle('Analisis Univariat Kolom-Kolom Kategorikal', fontsize=16, fontweight='bold', alpha=0.8, y=1.)
num_cat_cols = len(cat_cols)
num_rows = (num_cat_cols + 1) // 2  # Menghitung jumlah baris yang diperlukan untuk 2 kolom
for i in range(0, num_cat_cols):
    plt.subplot(num_rows, 2, i+1)   # Menggunakan 2 kolom
    sns.countplot(x=preped_train[cat_cols[i]], color='seagreen')
    plt.xticks(rotation=45, fontsize=8)  # Memutar label sumbu x sebesar 45 derajat dan mengatur fontsize
    plt.tight_layout()

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/analsis_kategorikal.png?raw=true)

In [ ]:
# Menampilkan baris dengan nilai yang kosong
mno.matrix(preped_train, figsize=(10, 5), fontsize=9)

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/missing_value.png?raw=true)

In [ ]:
# Menampilkan plot baris untuk variabel target `rain_1h`
plt.figure()
time_series=sns.lineplot(x=data_viz['year'], y="rain_1h", data=data_viz)
time_series.set_title("Curah Hujan")
time_series.set_ylabel("mm")

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/curah_hujan_1.png?raw=true)

In [ ]:
# Menampilkan plot regresi linear variabel target `rain_1h` untuk melihat trend 
time_series = sns.lmplot(x="year", y="rain_1h", data=data_viz, x_estimator=np.mean)
ax = time_series.axes[0, 0]
ax.set_title("Trend Curah Hujan")
plt.show()

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/curah_hujan_2.png?raw=true)

In [ ]:
# Menampilkan visualsisasi grafik garis fitur yang terkait dengan suhu ('temp', 'd_point', 'feels', 'min_temp', 'max_temp')
plt.figure()
time_series = sns.lineplot(x=data_viz['year'], y="temp", data=data_viz, label='temp')
time_series = sns.lineplot(x=data_viz['year'], y="d_point", data=data_viz, label='d_point')
time_series = sns.lineplot(x=data_viz['year'], y="feels", data=data_viz, label='feels')
time_series = sns.lineplot(x=data_viz['year'], y="min_temp", data=data_viz, label='min_temp')
time_series = sns.lineplot(x=data_viz['year'], y="max_temp", data=data_viz, label='max_temp')
time_series.set_title("Suhu")
time_series.set_ylabel("celcius")

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/suhu.png?raw=true)

In [ ]:
# Menampilkan visualisasi grafik garis fitur tekanan udara (prssr)
plt.figure()
time_series = sns.lineplot(x=data_viz['year'], y="prssr", data=data_viz, label='prssr')
time_series.set_title("Tekanan Udara")
time_series.set_ylabel("hPa")

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/tekanan_udara.png?raw=true)

In [ ]:
# Menampilkan visualisasi grafik garis fitur kecepatan angin (wind_spd)
time_series = sns.lineplot(x=data_viz['year'], y='wind_spd', data=data_viz, label='wind_spd')
time_series.set_title('Kecepatan Angin')
time_series.set_ylabel("m/s")

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/kecepatan_angin.png?raw=true)

In [ ]:
# Menampilkan visualisasi grafik garis fitur persentase kelembapan udara (hum)
plt.figure()
time_series = sns.lineplot(x=data_viz['year'], y='hum', data=data_viz, label='hum')
time_series.set_title("Kelembapan Udara")
time_series.set_ylabel("%")

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/kelembapan_udara.png?raw=true)

In [ ]:
# Menampilkan visualisasi grafik garis fitur persentase penutupan awan (clouds)
time_series = sns.lineplot(x=data_viz['year'], y='clouds', data=data_viz, label='clouds')
time_series.set_title('Awan')
time_series.set_ylabel("%")

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/awan.png?raw=true)

In [ ]:
# Untuk meringankan komputasi seaborn, maka kami ambil 3 tahun paling baru
data_viz_recent = data_viz[
    data_viz['year'] > 2014
]

print(data_viz_recent.shape)

In [ ]:
# Menampilkan visualisasi matriks pair plot untuk menganalisis hubungan antara beberapa variabel dalam dataset
# terutama fitur numerik dengan target variabel 'rain_1h'
To_Plot = ['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'clouds', 'rain_1h']
plt.figure()
sns.pairplot(data_viz_recent[To_Plot], diag_kind="kde", hue='rain_1h')
plt.show()

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/pair_plot_rain_1h.png?raw=true)

In [ ]:
# Menampilkan visualisasi matriks pair plot untuk menganalisis hubungan antara beberapa variabel dalam dataset
# terutama fitur numerik dengan fitur kategori 'hour_cat'
To_Plot = ['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'clouds', 'rain_1h', 'hour_cat']
plt.figure()
sns.pairplot(data_viz_recent[To_Plot], diag_kind="kde", hue='hour_cat') 
plt.show()

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/pair_plot_hour_cat.png?raw=true)

In [ ]:
# Menampilkan visualisasi matriks pair plot untuk menganalisis hubungan antara beberapa variabel dalam dataset
# terutama fitur numerik dengan fitur kategori 'visibility'
# Pertanyaan: Apakah terdapat pola data numerical dengan kategori `visibility`?
To_Plot = ['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'clouds', 'rain_1h', 'visibility']
plt.figure()
sns.pairplot(data_viz_recent[To_Plot], diag_kind="kde", hue='visibility') 
plt.show()

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/pair_plot_visibility.png?raw=true)

## Investigasi kolom tertentu

In [ ]:
data_viz.groupby(
    data_viz['hour_cat'])['rain_1h'].mean()

In [ ]:
data_viz.groupby(
    data_viz['dayofweek'])['rain_1h'].mean()

In [ ]:
data_viz.groupby(
    data_viz['quart_cat'])['rain_1h'].mean()

## ACF

In [ ]:
# Kode di bawah digunakan untuk menampilkan plot fungsi autokorelasi (Autocorrelation Function, ACF)
# Dari fungsi ACF ini kita dapat menggunakan nilai tersebut untuk lag

# Membuat daftar lags dari 1 hingga 60 (60 periode sebelumnya)
lags = list(range(1,61))

# Plot fungsi ACF untuk data kelembapan ("hum")
plot_acf(data_viz['hum'], lags=lags)
plt.title('Autocorrelation Function (ACF)')
plt.show()

![](https://github.com/radifan9/AIRNOLOGY-23-image/blob/main/acf.png?raw=true)

In [ ]:
# Kode di bawah berfungsi untuk menghitung nilai fungsi korelasi otokorelasi
acf_values = acf(data_viz['hum'])

corr_highest = [x for x, acf_value in enumerate(acf_values) if acf_value > 0.7 or acf_value < -0.35]  
corr_highest

## Informasi yang di dapat dari EDA

---
- Terdapat trend yang naik untuk curah hujan `rain_1h` dari tahun 1979-2017.
- Terdapat pembagian pola kategori yang jelas dari fitur `hour_cat`.
- Dari 4 quarter waktu, quarter ke 3 memiliki rata-rata curah hujan terendah --> bulan ke 7 sampai 9.
- Tidak ada pola yang dihasilkan dari kategori `visibility`.
- Dari plot ACF diketahui korelasi yang tinggi pada lag-lag sebelumnya.
- Outlier removal diperlukan karena pola data jauh lebih terlihat setelah dilakukan removal.
---

# Model Fitting

In [ ]:
# Train test split
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(
                                        transformed_train,
                                        target = 'rain_1h',
                                        train_size=0.7,
                                        random_state=42,
                                        )

# Memisahkan fitur dengan target
full_train = transformed_train.drop(columns=["rain_1h"])
full_target = transformed_train["rain_1h"]

# Fitting Model
tuned_cat_regressor = CatBoostRegressor()
start = time.time()
tuned_cat_regressor.fit(X_train, y_train, verbose=False)
end = time.time()
cat_y_pred = tuned_cat_regressor.predict(X_test)

# Evaluasi RMSE pada data split
cat_y_pred = tuned_cat_regressor.predict(X_test)
print('RMSE cat:', np.sqrt(mean_squared_error(y_test,  pd.DataFrame(cat_y_pred)[0].apply(lambda x: 0 if x < 0 else x))))

RMSE cat: 0.6886506012075394


In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = []

# cross validation dengan fold sejumlah 5
for idx, (train_index, valid_index) in enumerate(kf.split(full_train)):
    print(f"Fold {idx+1}")
    X_train, X_valid = full_train.iloc[train_index], full_train.iloc[valid_index]
    y_train, y_valid = full_target.iloc[train_index], full_target.iloc[valid_index]
    
    tuned_cat_regressor = CatBoostRegressor()
    tuned_cat_regressor.fit(X_train, y_train, verbose=False)
    
    cat_y_pred = tuned_cat_regressor.predict(X_valid)
    y_pred = pd.DataFrame(cat_y_pred)[0].apply(lambda x: 0 if x < 0 else x)
    print('RMSE cat:', np.sqrt(mean_squared_error(y_valid, y_pred )))
    scores.append(np.sqrt(mean_squared_error(y_valid, y_pred )))
    
    print()

Fold 1
RMSE cat: 0.6922487157771731

Fold 2
RMSE cat: 0.6906945182907925

Fold 3
RMSE cat: 0.7041806152751425

Fold 4
RMSE cat: 0.6885900491155985

Fold 5
RMSE cat: 0.6949747756530666



In [ ]:
np.mean(scores)

0.6941377348223545

In [ ]:
# Fitur paling penting
tuned_cat_regressor.get_feature_importance(prettified=True).sort_values(by='Importances', ascending=False).head(50)

,Feature Id,Importances
0,hour_cat_last_dawn,8.153180
1,hour_cat_last_late morning,4.225493
2,hummov_avg2,3.984948
3,hour_cat_last_noon,3.550039
4,hum_roc,3.508087
5,hum_percent_change_with_lag_1,2.312426
6,hour_cat_last_early morning,2.023064
7,temp_percent_change_with_lag_1,1.668399
8,hum_percent_change_with_lag_2,1.588632
9,hum_*_tempmov_avg2,1.566470


In [ ]:
# Fitur paling tidak penting
tuned_cat_regressor.get_feature_importance(prettified=True).sort_values(by='Importances', ascending=False).tail(50)

,Feature Id,Importances
1006,d_point_EMA_6,0.0
1007,hum_windmov_var28,0.0
1008,hum,0.0
1009,feels_EMA_3,0.0
1010,prssr_*_temp_EMA_3,0.0
1011,temp_wind_EMA_5,0.0
1012,clouds_*_wind_spd_EMA_4,0.0
1013,clouds_*_wind_spd_EMA_5,0.0
1014,tempmov_var28,0.0
1015,temp_wind_EMA_2,0.0


In [ ]:
# Refit dengan seluruh data latih
full_cat_regressor = CatBoostRegressor()
full_cat_regressor.fit(full_train, full_target, verbose=False)

# Kaggle Submission

In [ ]:
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission['rain_1h'] = full_cat_regressor.predict(transformed_test)
submission['rain_1h'] = submission['rain_1h'].apply(lambda x: 0. if x < 0 else x) # merubah semua rain_1h yang negatif menjadi 0, karena curah hujan selalu positif
submission.to_csv('../submission/submission_23.csv', index=False)

,datetime_iso,rain_1h
0,2018-01-01 00:00:00+00:00,0.984974
1,2018-01-01 01:00:00+00:00,1.309610
2,2018-01-01 02:00:00+00:00,0.594391
3,2018-01-01 03:00:00+00:00,0.151131
4,2018-01-01 04:00:00+00:00,0.433099
...,...,...
49363,2023-08-19 19:00:00+00:00,0.110330
49364,2023-08-19 20:00:00+00:00,0.669670
49365,2023-08-19 21:00:00+00:00,1.473739
49366,2023-08-19 22:00:00+00:00,1.303050


## Export

In [ ]:
import os
import pickle

SUBMISSION_KE = "submission_23"

# cek apakah folder sudah ada, jika belum maka dibuat folder submissionnya
if not os.path.exists(f'../besar/{SUBMISSION_KE}'):
    os.makedirs(f'../besar/{SUBMISSION_KE}')

# export train data
transformed_train.to_csv(
    f'../besar/{SUBMISSION_KE}/train_{SUBMISSION_KE}.csv', index=False)

# export test data
transformed_test.to_csv(
    f'../besar/{SUBMISSION_KE}/test_{SUBMISSION_KE}.csv', index=False)

# export model
pickle.dump(full_cat_regressor, open(f'../besar/{SUBMISSION_KE}/cat_regressor.pkl', 'wb'))

# Kesimpulan


Dataset ini menunjukkan adanya sejumlah pencilan (outlier) yang signifikan, yang mengharuskan pendekatan yang sangat teliti dalam proses pembersihan data. Dalam konteks ini, rekayasa fitur menjadi elemen kunci untuk mengurangi kesalahan Root Mean Square (RMSE), terutama melalui penerapan teknik dalam domain time series seperti rata-rata bergerak (moving average) dan lag. Penting untuk mencatat bahwa evaluasi kinerja model dengan pembagian data (data split) tidak selalu bersifat linear atau sejalan dengan nilai publik pada platform Kaggle. Oleh karena itu, penggunaan validasi silang (cross-validation) menyediakan estimasi yang lebih akurat terhadap performa model yang akan digunakan.